In [21]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import numpy as np
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, col, udf, rand
import pyspark.sql as ps
from pyspark.broadcast import Broadcast
import matplotlib.pyplot as plt
from zlib import crc32
import time as tm
from datetime import datetime as dt
import itertools
from dataclasses import dataclass
from pyspark.sql import functions as F
from pyspark.rdd import RDD

In [22]:
path = './data'
worker_nodes = "*"
problem_to_solve = 'CANCELLED'

dataset_limit = 1000
use_all_dataset_frames = True
number_of_splits = 10

dataset = 'yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018'
path = './data'

# DA SCRIVERE
- perche' usiamo i dataframe invece degli rdd nella prima parte
- aggiungere k fold cross validation
- aggiungere griglia parametri
- aggiungere label stratification
- aggiungere performance modello pyspark
- aggiungere check e info extra su dataset di base (es sbilanciamento)
- auroc, auprc, f1, 
- confronto con tree classifier

## Data Download

In [23]:
os.environ['KAGGLE_USERNAME'] = "davidetricella"
os.environ['KAGGLE_KEY'] = "e1ab3aae4a07f36b37a3a8bace74d9df"

In [24]:
def download_dataset():
    if not os.path.isdir(path):
        os.mkdir(path)
    if not os.listdir(path):
        try:
            api = KaggleApi()
            api.authenticate()
            api.dataset_download_files(dataset, path, unzip=True, quiet=False)
        except:
            print("Error downloading the dataset")

## Data Loading

In [ ]:
dataframe_schema = StructType([
    StructField('FL_DATE', StringType(), True),
    StructField('OP_CARRIER', StringType(), True),
    StructField('ORIGIN', StringType(), True),
    StructField('DEST', StringType(), True),
    StructField('CRS_DEP_TIME', StringType(), True),
    StructField('CRS_ARR_TIME', StringType(), True),
    StructField('CANCELLED', StringType(), True),
    StructField('DIVERTED', StringType(), True),
    StructField('CRS_ELAPSED_TIME', StringType(), True),
    StructField('DISTANCE', StringType(), True)
])

columns_to_get = [
    'FL_DATE',
    'OP_CARRIER',
    'ORIGIN',
    'DEST',
    'CRS_DEP_TIME',
    'CRS_ARR_TIME',
    'CANCELLED',
    'DIVERTED',
    'CRS_ELAPSED_TIME',
    'DISTANCE'
]

spark = SparkSession.builder \
.appName("Airline Departure") \
.master('local[' + worker_nodes + ']') \
.getOrCreate()

context = spark.sparkContext

In [ ]:
def load_dataset():
    data = spark.read.format("csv") \
        .option("header", True) \
        .load(path + '/preprocessed')

    print('Preprocessed dataset loaded')
    return data

def save_dataset(data):
    data.write.format('csv').option('header', True).mode('overwrite').option(
        'sep', ',').save(path + '/preprocessed')
    print('Preprocessed dataset saved')

def check_preprocessed_data_exists() -> bool:
    files = os.listdir('./data')
    for f in files:
        if f.startswith('preprocessed'):
            return True
    return False

def get_dataset(limit: float = -1, allFrames: bool = True):
    files = os.listdir(path)
    big_frame = spark.createDataFrame(
        spark.sparkContext.emptyRDD(), schema=dataframe_schema)
    if not allFrames:
        files = [files[0]]

    for f in files:
        if f.endswith('.csv'):
            frame = spark.read.option("header", True).csv(path + '/' + f)
            frame = frame.select(columns_to_get)
            frame = frame.sample(fraction=1.0, withReplacement=False)

            if limit != -1:
                frame = frame.limit(limit)

            big_frame = frame.union(big_frame)

    big_frame = big_frame.select(
        "*").withColumn("index", monotonically_increasing_id())
    big_frame.count()

    return big_frame


## Preprocessing

In [ ]:
default_values = {
    'CANCELLED': 0,
    'DIVERTED': 0
}

columns_to_remove_for_canceled = [
    'DIVERTED',  # the flight has been diverted to an unplanned airport
]

columns_to_remove_for_diverted = [
    'CANCELLED',  # the flight has been cancelled
]

names_columns_to_convert = [
    'OP_CARRIER',
    'ORIGIN',
    'DEST',
]

date_columns_to_convert = [
    'FL_DATE'
]

time_columns_to_convert = [
    'CRS_DEP_TIME',
    'CRS_ARR_TIME',
    'CRS_ELAPSED_TIME'
]

numeric_columns_to_convert = [
    'DISTANCE'
]

string_columns_to_convert = [
    'CANCELLED',
    'DIVERTED'
]

preprocess_columns_to_convert = [
    'OP_CARRIER',
    'ORIGIN',
    'DEST',
    'FL_DATE',
    'CRS_DEP_TIME',
    'CRS_ARR_TIME',
    'CRS_ELAPSED_TIME',
    'DISTANCE',
    'CANCELLED',
    'DIVERTED',
    'index'
]

max_distance = 4970

In [ ]:
def print_and_save_time(s: str):
    time_file.write(s + '\n')
    print(s)


def remove_extra_columns(index: str, data: ps.DataFrame) -> ps.DataFrame:

    start_time = tm.time()
    oppositeIndex = 'DIVERTED' if index == 'CANCELLED' else 'CANCELLED'
    data = data.drop(oppositeIndex)
    data = data.drop('index')
    data.count()

    finish_time = tm.time() - start_time
    print_and_save_time("Extra column removal concluded: " +
                        str(finish_time) + " seconds")
    return data


def convert_strings_into_numbers(data: ps.DataFrame) -> ps.DataFrame:
    udf_string_conversion = udf(lambda x: float(x), DoubleType())
    for c in string_columns_to_convert:
        data = data.withColumn(c, udf_string_conversion(col(c)))
    data.count()
    return data


def convert_names_into_numbers(data: ps.DataFrame) -> ps.DataFrame:

    def str_to_float(s: str):
        encoding = "utf-8"
        b = s.encode(encoding)
        return float(crc32(b) & 0xffffffff) / 2**32

    udf_names_conversion = udf(lambda x: str_to_float(x), DoubleType())
    for c in names_columns_to_convert:
        data = data.withColumn(c, udf_names_conversion(col(c)))
    data.count()
    return data


def convert_dates_into_numbers(data: ps.DataFrame) -> ps.DataFrame:
    multiplier: float = 1 / 365

    def date_to_day_of_year(date_string) -> float:

        date = dt.strptime(date_string, "%Y-%m-%d")
        day = date.timetuple().tm_yday - 1
        return day * multiplier

    udf_dates_conversion = udf(
        lambda x: date_to_day_of_year(x), DoubleType())
    for c in date_columns_to_convert:
        data = data.withColumn(c, udf_dates_conversion(col(c)))
    data.count()

    return data

def balance_dataframe(data: ps.DataFrame, index: str, n: int) -> ps.DataFrame:
    start_time = tm.time()
    positives = data.filter(col(index) == 1)
    negatives = data.filter(col(index) == 0)

    result = positives.limit(n).\
        union(negatives.limit(n)).\
        orderBy(rand())
    finish_time = tm.time() - start_time
    print_and_save_time("Dataset balancing concluded: " +
                        str(finish_time) + " seconds")
    return result

def convert_times_into_numbers(data: ps.DataFrame) -> ps.DataFrame:

    def time_to_interval(time) -> float:
        t = int(float(time))
        h = t // 100
        m = t % 100
        t = h * 60 + m
        return float(t / 1140)

    udf_time_conversion = udf(lambda x: time_to_interval(x), DoubleType())
    for c in time_columns_to_convert:
        data = data.withColumn(c, udf_time_conversion(col(c)))
    data.count()

    return data


def convert_distance_into_numbers(data: ps.DataFrame) -> ps.DataFrame:

    multiplier: float = float(1) / float(max_distance)
    udf_numeric_conversion = udf(
        lambda x: float(x) * multiplier, DoubleType())

    data = data.withColumn(
        'DISTANCE', udf_numeric_conversion(col('DISTANCE')))
    data.count()
    return data


def split_data(data: ps.DataFrame, label: str, k: int) -> tuple[ps.DataFrame, ps.DataFrame]:
    split_list = []
    data = data.dropDuplicates()
    print(data.count())

    total_positives = data.filter(col(label) == 1).count()
    total_negatives = data.filter(col(label) == 0).count()
    positives_negatives_ratio = total_positives/total_negatives
    k_elements_number = round(data.count() / k)

    k_positive_elements = round(
        k_elements_number * positives_negatives_ratio)
    k_negative_elements = round(
        k_elements_number * (1 - positives_negatives_ratio))

    i = 0
    while i < k:
        k_positive_sample = data.where(
            col(label) == 1).limit(k_positive_elements)
        k_negative_sample = data.where(
            col(label) == 0).limit(k_negative_elements)
        k_sample = k_positive_sample.union(k_negative_sample)
        
        print(k_sample.count())
        split_list.append(np.array(k_sample.collect()))
        data = data.subtract(k_sample)

        print("Split " + str(i + 1) + " of " + str(k) + " completed")
        i += 1

    return split_list

def common_preprocess(data: ps.DataFrame) -> ps.DataFrame:

    common_start_time = tm.time()

    # Remove rows with Nan key values
    data = data.dropna(how='any')

    null_removal_finish_time = tm.time() - common_start_time
    print_and_save_time("Null values removal concluded: " +
                        str(null_removal_finish_time) + " seconds")

    data = balance_dataframe(data, problem_to_solve, dataset_limit)

    names_start_time = tm.time()
    data = convert_names_into_numbers(data)
    names_finish_time = tm.time() - names_start_time
    print_and_save_time("Names conversion concluded: " +
                        str(names_finish_time) + " seconds")

    dates_start_time = tm.time()
    data = convert_dates_into_numbers(data)
    dates_finish_time = tm.time() - dates_start_time
    print_and_save_time("Dates conversion concluded: " +
                        str(dates_finish_time) + " seconds")

    times_start_time = tm.time()
    data = convert_times_into_numbers(data)
    times_finish_time = tm.time() - times_start_time
    print_and_save_time("Times conversion concluded: " +
                        str(times_finish_time) + " seconds")

    distance_start_time = tm.time()
    data = convert_distance_into_numbers(data)
    distance_finish_time = tm.time() - distance_start_time
    print_and_save_time("Distance conversion concluded: " +
                        str(distance_finish_time) + " seconds")

    strings_start_time = tm.time()
    data = convert_strings_into_numbers(data)
    strings_finish_time = tm.time() - strings_start_time
    print_and_save_time("Strings conversion concluded: " +
                        str(strings_finish_time) + " seconds")

    common_finish_time = tm.time() - common_start_time
    print_and_save_time("Common preprocessing concluded: " +
                        str(common_finish_time) + " seconds")
    return data


#PREPROCESSING EXECUTION
time_file = open("./data/times.txt", "w")

if not check_preprocessed_data_exists():
        download_dataset()

        start_time = tm.time()
        data = get_dataset(dataset_limit, use_all_dataset_frames)

        finish_time = tm.time() - start_time
        print_and_save_time("Dataset reading concluded: " +
                            str(finish_time) + " seconds")

        data = common_preprocess(data)
        save_dataset(data)
else:
    data = load_dataset()
    udf_string_conversion = udf(lambda x: float(x), DoubleType())
    for c in preprocess_columns_to_convert:
        data = data.withColumn(c, udf_string_conversion(col(c)))
    
data = remove_extra_columns(problem_to_solve, data)

start_time = tm.time()
preprocessing_splits = split_data(data, problem_to_solve, number_of_splits)

finish_time = tm.time() - start_time
print_and_save_time("Dataset splitting concluded: " +
                    str(finish_time) + " seconds")

# Preprocess

## Da aggiungere Z score normalization

Bonus: Pandas

In [ ]:
def pandas_load_dataset():
    data = pd.read_csv(filepath_or_buffer=path + '/' + 'preprocessed.csv')
    print('Preprocessed dataset loaded')
    return data

def pandas_save_dataset(data):
    data.to_csv(path_or_buf=path + '/' + 'preprocessed.csv', index=False)
    print('Preprocessed dataset saved')

def pandas_get_dataset(limit: float = -1, allFrames: bool = True):
    files = os.listdir(path)
    big_frame = pd.DataFrame()

    if not allFrames:
        files = [files[0]]

    for f in files:
        if f.endswith('.csv'):
            frame = pd.read_csv(filepath_or_buffer=path +
                                '/' + f, usecols=columns_to_get)
            if limit != -1:
                frame = frame.sample(n=limit, replace=False)
            big_frame = pd.concat([big_frame, frame])
    return big_frame


def pandas_remove_extra_columns(index: str, data: pd.DataFrame) -> pd.DataFrame:

    start_time = tm.time()
    oppositeIndex = 'DIVERTED' if index == 'CANCELLED' else 'CANCELLED'
    data.drop(oppositeIndex, axis=1, inplace=True)

    finish_time = tm.time() - start_time
    print_and_save_time("Extra column removal concluded: " +
                        str(finish_time) + " seconds")
    return data


def pandas_balance_dataframe(data: pd.DataFrame, index: str, n: int) -> pd.DataFrame:
    start_time = tm.time()
    positives = data.query(index + ' == 1')
    negatives = data.query(index + ' == 0')
    
    result = pd.concat([positives.sample(n), negatives.sample(n)]).sample(frac=1)
    finish_time = tm.time() - start_time
    print_and_save_time("Dataset balancing concluded: " +
                        str(finish_time) + " seconds")
    return result


def pandas_convert_names_into_numbers(data: pd.DataFrame) -> pd.DataFrame:

    def str_to_float(s: str):
        encoding = "utf-8"
        b = s.encode(encoding)
        return float(crc32(b) & 0xffffffff) / 2**32

    for c in names_columns_to_convert:
        data[c] = data[c].apply(str_to_float)
    return data


def pandas_convert_dates_into_numbers(data: pd.DataFrame) -> pd.DataFrame:
    multiplier: float = 1 / 365

    def date_to_day_of_year(date_string) -> float:
        date = dt.strptime(date_string, "%Y-%m-%d")
        day = date.timetuple().tm_yday - 1
        return day * multiplier

    for i in date_columns_to_convert:
        data[i] = data[i].apply(date_to_day_of_year)

    return data


def pandas_convert_times_into_numbers(data: pd.DataFrame) -> pd.DataFrame:

    def time_to_interval(time) -> float:
        t = int(float(time))
        h = t // 100
        m = t % 100
        t = h * 60 + m
        return float(t / 1140)

    for c in time_columns_to_convert:
        data[c] = data[c].apply(time_to_interval)

    return data


def pandas_convert_distance_into_numbers(data: pd.DataFram) -> pd.DataFrame:

    multiplier: float = float(1) / float(max_distance)

    for c in numeric_columns_to_convert:
        data[c] = data[c].apply(lambda x: x * multiplier)
    return data


def pandas_split_data(data: pd.DataFrame, label: str, k: int) -> tuple[pd.DataFrame, pd.DataFrame]:
    split_list = []

    data.drop_duplicates(inplace=True)

    data_positives = data.query(label + ' == 1')
    data_negatives = data.query(label + ' == 0')

    total_positives = len(data_positives)
    total_negatives = len(data_negatives)
    positives_negatives_ratio = total_positives/total_negatives

    k_elements_number = round(len(data) / k)

    k_positive_elements = round(
        k_elements_number * positives_negatives_ratio)
    k_negative_elements = round(
        k_elements_number * (1 - positives_negatives_ratio))

    for i in range(1, k + 1):
        k_positive_sample = data_positives.head(k_positive_elements)
        k_negative_sample = data_negatives.head(k_negative_elements)
        k_sample = pd.concat([k_positive_sample, k_negative_sample])

        split_list.append(k_sample.to_numpy())
        data_positives = data_positives.drop(k_positive_sample.index)
        data_negatives = data_negatives.drop(k_negative_sample.index)

    return split_list

def pandas_common_preprocess(data: ps.DataFrame | pd.DataFrame) -> ps.DataFrame | pd.DataFrame:

    common_start_time = tm.time()

    data.dropna(how='any', axis='index', inplace=True)

    null_removal_finish_time = tm.time() - common_start_time
    print_and_save_time("Null values removal concluded: " +
                        str(null_removal_finish_time) + " seconds")

    pandas_balance_dataframe(data, problem_to_solve, dataset_limit)

    names_start_time = tm.time()
    data = pandas_convert_names_into_numbers(data)
    names_finish_time = tm.time() - names_start_time
    print_and_save_time("Names conversion concluded: " +
                        str(names_finish_time) + " seconds")

    dates_start_time = tm.time()
    data = pandas_convert_dates_into_numbers(data)
    dates_finish_time = tm.time() - dates_start_time
    print_and_save_time("Dates conversion concluded: " +
                        str(dates_finish_time) + " seconds")

    times_start_time = tm.time()
    data = pandas_convert_times_into_numbers(data)
    times_finish_time = tm.time() - times_start_time
    print_and_save_time("Times conversion concluded: " +
                        str(times_finish_time) + " seconds")

    distance_start_time = tm.time()
    data = pandas_convert_distance_into_numbers(data)
    distance_finish_time = tm.time() - distance_start_time
    print_and_save_time("Distance conversion concluded: " +
                        str(distance_finish_time) + " seconds")

    common_finish_time = tm.time() - common_start_time
    print_and_save_time("Common preprocessing concluded: " +
                        str(common_finish_time) + " seconds")
    return data

if not check_preprocessed_data_exists():
        download_dataset()

        start_time = tm.time()
        data = get_dataset(dataset_limit, use_all_dataset_frames)

        finish_time = tm.time() - start_time
        print_and_save_time("Dataset reading concluded: " +
                            str(finish_time) + " seconds")

        data = pandas_common_preprocess(data)
        pandas_save_dataset(data)
else:
    data = pandas_load_dataset()
    
data = pandas_remove_extra_columns(problem_to_solve, data)

start_time = tm.time()
preprocessing_splits = pandas_split_data(data, problem_to_solve, number_of_splits)

finish_time = tm.time() - start_time
print_and_save_time("Dataset splitting concluded: " +
                    str(finish_time) + " seconds")

## Models

### Generic Functions

In [2]:
def sigmoid(x):
    '''
    Calculates the sigmoid of the given data
    '''
    g = 1.0 / (1.0 + np.exp(-x))
    return g

def binary_cross_entropy(y, y_label, w, l2):
    '''
    Calculates the binary cross entropy loss of the calculated y and the given y_label
    '''
    loss = -np.mean(y_label*(np.log(y)) + (1-y_label)
                    * np.log(1-y)) + regularize(w, l2)
    return loss

def regularize(W, l2):
    '''
    Calculates the regularization term for the loss
    '''
    return (l2 / 2) * np.sum(np.square(W))

### Parallel Model

In [ ]:
@dataclass
class ParallelLogisticRegression:
    iterations: int
    learning_rate: float
    batch_size: int
    l2: float
    W: Broadcast
    b: float

def initialize(self: ParallelLogisticRegression, size):
    self.W = context.broadcast(np.random.rand(size))
    self.b = np.random.rand()


def train(self: ParallelLogisticRegression, data: ps.DataFrame):
    initialize(self, len(data.columns) - 1)

    num_chunks = X.count() // self.batch_size
    chunk_percent = 1/num_chunks

    batches = data.randomSplit([chunk_percent] * num_chunks)

    Y_labels = [b.select(problem_to_solve).rdd for b in batches]
    X = [b.drop(problem_to_solve).rdd for b in batches]

    losses = []
    gradients = []

    for _ in range(self.iterations):
        _losses = []
        _gradients = [] 

        for b_X, b_Y_labels in zip(X, Y_labels):
            Y = self.evaluate(b_X)
            _losses.append(binary_cross_entropy(
                Y, b_Y_labels, self.W, self.l2))
            (dW, db) = self.gradient(b_X, Y, b_Y_labels)
            _gradients.append(dW)
            self.update(dW, db)
        losses.append(np.mean(_losses))
        gradients.append(np.mean(_gradients))

        return (losses, gradients)

def evaluate(self: ParallelLogisticRegression, X: RDD):
    Z = X.map(lambda x: np.dot(x, self.W.value)).reduce(lambda a, b: a+b + self.b)
    Z = Z.map(lambda x: sigmoid(x))
    return Z
        
def update(self, dW: list[float], db: float):
        self.W = context.broadcast(self.W.value - self.learning_rate * dW)
        self.b = self.b - self.learning_rate * db

### Serial Model

In [ ]:
class SerialLogisticRegression():
    def __init__(self, iterations: int, learning_rate: float, batch_size: int, l2: float):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.l2 = l2

    def initialize(self, columns_number):
        self.W = np.random.rand(columns_number)
        self.b = np.random.rand()

    def evaluate(self, X):
        Z = np.dot(X, self.W) + self.b
        Z = sigmoid(Z)
        return Z

    def gradient(self, X, Y, Y_label):
        '''
        Calculates the gradient w.r.t weights and bias
        '''

        # Number of training examples.
        m = X.shape[0]

        # Gradient of loss w.r.t weights with regularization
        dw = (1/m)*np.dot(X.T, (Y - Y_label)) + self.l2 * self.W

        # Gradient of loss w.r.t bias with regularization
        db = (1/m)*np.sum((Y - Y_label))

        return dw, db

    def update(self, dW, db):
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db

    def train(self, X, Y_labels):
        self.initialize(X.shape[1])
        losses = []
        gradients = []

        for _ in range(self.iterations):
            _losses = []
            _gradients = []
            for b in range(X.shape[0]//self.batch_size):
                b_X = X[b*self.batch_size:b*self.batch_size+self.batch_size, :]
                b_Y_labels = Y_labels[b*self.batch_size:b *
                                      self.batch_size+self.batch_size]
                Y = self.evaluate(b_X)
                _losses.append(binary_cross_entropy(
                    Y, b_Y_labels, self.W, self.l2))
                (dW, db) = self.gradient(b_X, Y, b_Y_labels)
                _gradients.append(dW)
                self.update(dW, db)
            losses.append(np.mean(_losses))
            gradients.append(np.mean(_gradients))

        return (losses, gradients)

## Experiments


In [ ]:
def make_roc(labels, results, name):
    labels_and_results = sorted(
        list(zip(labels, map(lambda x: x, results))), key=lambda x: x[1])

    labels_by_weights = np.array([k for (k, _) in labels_and_results])

    length = labels_by_weights.size

    true_positives = labels_by_weights.cumsum()

    num_positive = true_positives[-1]

    false_positives = np.arange(1.0, length + 1, 1.) - true_positives

    true_positives_rate = true_positives / num_positive
    false_positives_rate = false_positives / (length - num_positive)

    fig, ax = plt.subplots()
    ax.set_xlim(-.05, 1.05), ax.set_ylim(-.05, 1.05)
    ax.set_ylabel('True Positive Rate (Sensitivity)')
    ax.set_xlabel('False Positive Rate (1 - Specificity)')
    plt.plot(false_positives_rate, true_positives_rate,
             color='#8cbfd0', linestyle='-', linewidth=3.)
    plt.plot((0., 1.), (0., 1.), linestyle='--',
             color='#d6ebf2', linewidth=2.)

    plt.savefig('./data/{}_roc.png'.format(name))
    fig.clear()
    plt.close()

def plot_loss_gradient(iterations, train_losses, gradients, name):
    fig, ax = plt.subplots()
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Loss/Gradient')
    ax.set_title(name)
    ax.plot(range(iterations), train_losses, label='Loss')
    ax.plot(range(iterations), gradients, label='Gradient')
    ax.grid()
    ax.legend()

    fig.savefig("./data/{}.png".format(name))
    fig.clear()
    plt.close()

### Hyperparamters Tuning

In [12]:
grid = { 'iter': [100, 200, 500], 'lr': [0.001, 0.01, 0.1], 'l2': [0, 0.1, 0.001], 'batch_size': [0, 20]}

params = list(itertools.product(*grid.values()))

params

[(100, 0.001, 0),
 (100, 0.001, 0.1),
 (100, 0.001, 0.001),
 (100, 0.01, 0),
 (100, 0.01, 0.1),
 (100, 0.01, 0.001),
 (100, 0.1, 0),
 (100, 0.1, 0.1),
 (100, 0.1, 0.001),
 (200, 0.001, 0),
 (200, 0.001, 0.1),
 (200, 0.001, 0.001),
 (200, 0.01, 0),
 (200, 0.01, 0.1),
 (200, 0.01, 0.001),
 (200, 0.1, 0),
 (200, 0.1, 0.1),
 (200, 0.1, 0.001),
 (500, 0.001, 0),
 (500, 0.001, 0.1),
 (500, 0.001, 0.001),
 (500, 0.01, 0),
 (500, 0.01, 0.1),
 (500, 0.01, 0.001),
 (500, 0.1, 0),
 (500, 0.1, 0.1),
 (500, 0.1, 0.001)]

In [ ]:

class Model:
    def __init__(self, iterations, lr, l2, batch_size):
        self.iterations = iterations
        self.lr = lr
        self.l2 = l2
        self.batch_size = batch_size

    def train(self, data):
        total_train_losses = []
        total_test_losses = []
        for i, fold in enumerate(data):
            train_data = data[:i] + data[i + 1:]
            test_data = fold
            losses,  = self.train_impl(train_data)
            total_train_losses.append(losses) 
            loss = self.test_impl(test_data)
            total_test_losses.append(loss)
        return total_train_losses, total_test_losses




